# PathoAssist: The "Under-the-Hood" Tour \U0001f680

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

Welcome to the backend engine room of **PathoAssist**. While our interactive desktop app provides a seamless, "Apple-style" clinical experience, this notebook peels back the UI to show you exactly how our pipeline works.

We will walk through the 4 core steps of the PathoAssist backend as they happen in the app:
1. **The Viewer Phase:** Ingesting a histological sample.
2. **The Preprocessing Phase:** Our smart Otsu thresholding that discards useless glass background.
3. **The Inference Engine:** Loading MedGemma with clinical RAG on edge hardware.
4. **The PathoCopilot:** Simulating our interactive chatbot layer.

> **Note:** Ensure you are running this in a GPU runtime (Runtime > Change runtime type > T4 GPU) to simulate "Edge" consumer hardware.

In [ ]:
# Install core backend dependencies
!pip install -q torch transformers pillow opencv-python matplotlib "accelerate>=0.26.0" bitsandbytes huggingface_hub

### Authentication (MedGemma Access)
Google's MedGemma models are gated. 
1. Ensure you have accepted the terms on [google/medgemma-2b](https://huggingface.co/google/medgemma-2b).
2. Run the cell below to securely log in with your HuggingFace User Access Token.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

---
## Step 1: The Viewer Phase (WSI Ingestion)
In the PathoAssist app, a clinician drags and drops a massive gigapixel `.svs` WSI file. Our backend slices this into manageable tiles. 

For this interactive tour, we will download a **Real Clinical Sample** (an H&E stained histological patch of Colorectal Carcinoma) directly from a verified medical database.

In [ ]:
import torch
import numpy as np
import cv2
import urllib.request
import matplotlib.pyplot as plt
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText

# 1. Download an Actual Tissue Patch (Colorectal Carcinoma H&E Stain)
image_url = "https://upload.wikimedia.org/wikipedia/commons/e/ec/Colorectal_carcinoma%2C_H%26E_stain.jpg"
req = urllib.request.Request(image_url, headers={'User-Agent': 'Mozilla/5.0'})
with urllib.request.urlopen(req) as response:
    with open("clinical_patch.jpg", "wb") as f:
        f.write(response.read())

raw_image = Image.open("clinical_patch.jpg").convert("RGB")

plt.figure(figsize=(6, 6))
plt.imshow(raw_image)
plt.title("Step 1: Raw Clinical Tissue Patch (H&E)")
plt.axis('off')
plt.show()

---
## Step 2: Smart Tissue Detection (Preprocessing)
WSIs contain up to 60% blank whitespace (glass). Feeding glass to MedGemma is a massive waste of compute resource. 

Here, we demonstrate the exact `tiling.py` logic used in PathoAssist. We apply **Otsu's Thresholding** to create a binary mask, proving mathematically to the engine whether the patch is worth analyzing.

In [ ]:
def demonstrate_tissue_detection(image: Image.Image):
    img_array = np.array(image)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    
    # PathoAssist Core Logic: Otsu's Thresholding
    # Automatically finds the optimal separation between dark tissue and bright glass
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    tissue_pixels = np.sum(binary < 200)
    tissue_ratio = tissue_pixels / binary.size
    
    # Visualizing the "Under the Hood" Process
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.imshow(gray, cmap='gray')
    ax1.set_title("Grayscale Conversion")
    ax1.axis('off')
    
    # Invert binary for visualization (White = Human Tissue)
    mask_viz = np.where(binary < 200, 255, 0)
    ax2.imshow(mask_viz, cmap='magma')
    ax2.set_title(f"AI Tissue Mask (Density: {tissue_ratio*100:.1f}%)")
    ax2.axis('off')
    plt.show()

    if tissue_ratio > 0.15:
        print("\u2705 Status: Valid Tissue. Pushing to MedGemma.")
    return tissue_ratio

density = demonstrate_tissue_detection(raw_image)

---
## Step 3: The Inference Engine (MedGemma)
Now we load Google's HAI-DEF MedGemma model. 

**The Edge Feasibility Test:** To prove PathoAssist works in rural clinics without expensive servers, we load the model in **4-bit dynamic quantized mode**. This shrinks a massively powerful LLM so it fits comfortably inside standard laptop RAM (simulated here by Colab's T4 GPU).

In [ ]:
MODEL_ID = "google/medgemma-2b" 

print("Loading MedGemma via bitsandbytes (4-bit)...\n")
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID, 
    device_map="auto",
    load_in_4bit=True, # Crucial for Edge deployment
    torch_dtype=torch.float16
)
print("\u2714\ufe0f Engine Initialized on Edge equivalent hardware.")

### Clinical RAG (Retrieval-Augmented Generation)
PathoAssist doesn't just do image classification. It does **Clinical Reasoning**. We inject the patient's EHR (Electronic Health Record) directly into the multimodal payload, forcing MedGemma to interpret the image *through the lens* of the patient's history.

In [ ]:
def analyze_patch_with_rag(processor, model, image, clinical_context):
    img_resized = image.resize((224, 224), Image.Resampling.LANCZOS)
    
    # Replicating backend/app/inference/engine.py payload construction
    text_prompt = f"Based on this pathology patch and the patient's clinical history: ({clinical_context}), describe the morphological findings. Be specific and act as an expert pathologist."
    
    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": text_prompt}
        ]}
    ]
    text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=text, images=[img_resized], return_tensors="pt").to(model.device)
    
    print("\u2699\ufe0f Running Multimodal Inference...")
    with torch.no_grad():
        generation = model.generate(**inputs, max_new_tokens=256, temperature=0.2, do_sample=True)
    
    # Decode response
    generated_tokens = generation[0][inputs['input_ids'].shape[1]:]
    report = processor.decode(generated_tokens, skip_special_tokens=True)
    
    return report, messages # We return messages to maintain chat state for Step 4

mock_ehr = "Patient: 62M. History of altered bowel habits and weight loss. Colonoscopy revealed a fungating mass in the ascending colon. Biopsy performed."
print(f"\n\U0001f4cb Injected EHR Context: {mock_ehr}\n")

ai_report, chat_history = analyze_patch_with_rag(processor, model, raw_image, mock_ehr)

print("===============================")
print("\u2728 PathoAssist Preliminary Report \u2728")
print("===============================")
print(ai_report)

---
## Step 4: The PathoCopilot (Interactive Chat)
The most powerful UI feature of PathoAssist is the interactive Copilot. If a doctor disagrees or wants clarification, they don't have to guess. They can debate the model. 

Because we maintain state in our backend architecture, the model remembers the image and its previous report.

In [ ]:
def simulate_copilot_interaction(processor, model, chat_history, image, doctor_question):
    img_resized = image.resize((224, 224), Image.Resampling.LANCZOS)
    
    # Append the AI's previous answer to the history
    chat_history.append({"role": "assistant", "content": ai_report})
    
    # Append the Doctor's new question
    chat_history.append({"role": "user", "content": [{"type": "text", "text": doctor_question}]})
    
    text = processor.apply_chat_template(chat_history, add_generation_prompt=True)
    inputs = processor(text=text, images=[img_resized], return_tensors="pt").to(model.device)
    
    print(f"\U0001f468\u200d\u2695\ufe0f Doctor asks: \"{doctor_question}\"\n")
    print("\u2699\ufe0f Copilot is thinking...\n")
    
    with torch.no_grad():
        generation = model.generate(**inputs, max_new_tokens=256, temperature=0.3, do_sample=True)
        
    generated_tokens = generation[0][inputs['input_ids'].shape[1]:]
    answer = processor.decode(generated_tokens, skip_special_tokens=True)
    
    print("\U0001f916 Copilot Replies:")
    print(answer)

# Simulating a real clinical workflow interaction
simulate_copilot_interaction(
    processor, 
    model, 
    chat_history, 
    raw_image, 
    "Are there any architectural patterns like irregular glands or mucin production visible here?"
)